In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import obspy as obs
from obspy.clients.fdsn.client import Client
from obspy.geodetics.base import gps2dist_azimuth
from obspy.geodetics import kilometers2degrees
from obspy.taup import TauPyModel
from obspy.core.utcdatetime import UTCDateTime
import pandas as pd
import os
from scipy import signal
%matplotlib inline

In [2]:
############# PARAMETERS #############
mseed_dir = '/Users/rshimony/Desktop/WillametteValley/salem_eq/mseed/'
fig_dir = '/Users/rshimony/Desktop/WillametteValley/salem_eq/figs/'
meta_dir = '/Users/rshimony/Desktop/WillametteValley/salem_eq/metadata/'

## geographic region:
minlat = 43.4
maxlat = 46.7
minlon = -124.2
maxlon = -121.5

## OR central longitude/latitude and radius:
central_lon = -123.05
central_lat = 44.8
search_radius_min = 0 ## in degrees
search_radius_max = 2.5 ## in degrees

## time constraints for event search:
eventTime = UTCDateTime("2022-10-07T12:52:35.448127Z")
stime = eventTime - 60  # 1 minute before the event
etime = eventTime + 15*60  # 15 minutes after the event
# stime = UTCDateTime("2000-01-01T000000.000")
# etime = UTCDateTime("2021-9-20T235959.000")

## min depth in km:
min_depth = 1

## stations of interest:
netcode = '??'
# all_stations = 'BUCK'
channels = '???' # high gain accelerometers, could do high gain broadbands ('HH*',)
channel_split = channels.split(',')
## NOTE: BACME only has HN (W1 loc code); NCBC has

## CLIENTS
## Set up the client to be IRIS, for downloading:
client = Client('IRIS')
# taup:
model = TauPyModel(model="iasp91")

In [3]:
############# CATALOG DOWNLOADS ##############

## Find stations metadata for position:
sta_inventory = client.get_stations(network=netcode, channel=channels, starttime=stime, endtime=etime, minlatitude = minlat , maxlatitude = maxlat , 
                                      minlongitude = minlon , maxlongitude = maxlon,level="channel")

## Find earthquakes 
#eq_catalog = client.get_events(starttime=stime, endtime=etime,
#                        minlatitude=minlat, maxlatitude=maxlat, 
#                        minlongitude=minlon, maxlongitude=maxlon,
#                        minmagnitude=3)

eq_catalog = client.get_events(starttime=stime, endtime=etime,
                        latitude=central_lat, longitude=central_lon, 
                        minradius=search_radius_min, maxradius=search_radius_max,
                        minmagnitude=3,mindepth=min_depth)

In [18]:
# print(sta_inventory[3][2].channels[12].code)
# print(sta_inventory[3][2].channels[12].sample_rate)
# print(len(sta_inventory))

sta_inventory[8][5]

Station CARM (McKenzie Bridge, OR, USA)
	Station Code: CARM
	Channel Count: 17/23 (Selected/Total)
	2017-05-04T00:00:00.000000Z - 
	Access: open 
	Latitude: 44.30, Longitude: -122.04, Elevation: 991.0 m
	Available Channels:
		CARM..HNZ, CARM..HNN, CARM..HNE, CARM.D0.GAN, CARM.D0.GEL, 
		CARM.D0.GLA, CARM.D0.GLO, CARM.D0.GNS, CARM.D0.GPL, CARM.D0.GST, 
		CARM.D0.LCE, CARM.D0.LCQ, CARM.D0.VCO, CARM.D0.VDT, CARM.D0.VEC, 
		CARM.D0.VEI, CARM.D0.VPB

In [22]:
st = Client.get_waveforms(network="UO", station="CARM", location='*',channel='???',
                          starttime=stime, endtime=etime,attach_response=True)

TypeError: get_waveforms() missing 1 required positional argument: 'self'

In [245]:
# print(sta_inventory[1].code)
# print(channel_split[1][1:-1])
# st.code

# sta_inventory[0][0].elevation

evid_str=str(eq_catalog[35].resource_id)
evid_split = evid_str.split('/')[-1]
ev_id = evid_split.split('=')[-1]
print(evid_str)
print(ev_id)

smi:service.iris.edu/fdsnws/event/1/query?eventid=5157611
5157611


In [6]:
######## GET CATALOG AND METADATA ###########
## Extract the positions of the infrasound stations:
nt_stas = []
st_lons = []
st_lats = []
st_stas = []
st_start_year = []
st_elev = []
st_chan = []
ch_samprt = []
for network in sta_inventory:
    for station in network:
        for channel in station:
            if channel.sample_rate > 21:
                nt_stas.append(network.code)
                st_stas.append(station.code)
                st_lons.append(station.longitude)
                st_lats.append(station.latitude)
                st_start_year.append(station.start_date.year)
                st_elev.append(station.elevation)
                st_chan.append(channel.code)
                ch_samprt.append(channel.sample_rate)
                print(f'{channel.code[0:-1]}N , {channel.code[0:-1]}2')
stdict = {'network':nt_stas, 'name':st_stas, 'longitude':st_lons,
          'latitude':st_lats, 'elevation':st_elev, 'channel':st_chan, 'samprt':ch_samprt }
stdf = pd.DataFrame(stdict)
stdf_nodup=stdf.drop_duplicates()
stdf_nodup.to_csv(meta_dir + 'station_inventory.csv',index=False)
        
# Extract event information:
ev_lons = []
ev_lats = []
ev_depth = []
ev_origint = []
ev_mag = []
ev_id = []
for event in eq_catalog:
    ev_lons.append(event.origins[0].longitude)
    ev_lats.append(event.origins[0].latitude)
    ev_depth.append(event.origins[0].depth)
    ev_origint.append(event.origins[0].time)
    ev_mag.append(event.magnitudes[0].mag)
    evid_str=str(event.resource_id)
    evid_split = evid_str.split('/')[-1]
    evid = evid_split.split('=')[-1]
    ev_id.append(evid)
    
## WRite out event metadata to file to use later:
evdict = {'catalog #':ev_id, 'longitude':ev_lons, 'latitude':ev_lats,'magnitude':ev_mag,
          'depth':ev_depth,'time':ev_origint}
evdf = pd.DataFrame(evdict)
evdf.to_csv(meta_dir + 'event_catalog.csv',index=False)

BHN , BH2
BHN , BH2
BHN , BH2
BDN , BD2
HDN , HD2
HDN , HD2
HDN , HD2
HDN , HD2
HDN , HD2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BDN , BD2
BDN , BD2
BDN , BD2
BHN , BH2
BHN , BH2
BHN , BH2
HHN , HH2
HHN , HH2
HHN , HH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BDN , BD2
BHN , BH2
BHN , BH2
BHN , BH2
BDN , BD2
BDN , BD2
BDN , BD2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BDN , BD2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BHN , BH2
BDN , BD2
HNN , HN2
HNN , HN2
HNN , HN2
HNN , HN2
HNN , HN2
HNN , HN2
HNN , HN2
HNN , HN2
HNN , HN2
HNN , HN2
HNN , HN2
HNN , HN2
HNN , HN2


In [7]:
######## get distances btwn stations and events ###########
## make dataframe with station repeated, use this info for dl-ing waveforms
distances_all = []
sta_df_all = []
ev_mag_df_all = []
ev_origintime_df_all = []
ev_collecttime_all = []
ev_endtime_all = []
evdf_lat_all = []
evdf_lon_all = []
evdf_depth_all = []



for i_station in range(len(st_stas)):
    i_stlon = st_lons[i_station]
    i_stlat = st_lats[i_station]
    for event in eq_catalog:    
        ## add station name to long array:
        sta_df_all.append(st_stas[i_station])
        ## append event info:
        evdf_lat_all.append(event.origins[0].latitude)
        evdf_lon_all.append(event.origins[0].longitude)
        evdf_depth_all.append(event.origins[0].depth)
        
        ## get great circle distance between this event, and station:
        ij_grcircle_distance = gps2dist_azimuth(i_stlat, i_stlon, event.origins[0].latitude, event.origins[0].longitude)
        ij_grcircle_distance_km = ij_grcircle_distance[0]/1000
        ij_degrees = kilometers2degrees(ij_grcircle_distance_km)
        #ij_ttime = IRISclient.traveltime(model='iasp91', phases=['p'], evdepth=event.origins[0].depth, distkm=[ij_grcircle_distance_km], traveltimeonly=True)
        ## For each distance, assume 5 km/s average p-wave speed to get p-wave travel time:
        #ij_pwv_ttime = ij_grcircle_distance_km / 5.
        ij_pwv_ttime_allP = model.get_travel_times(source_depth_in_km=event.origins[0].depth/1000,
                                  distance_in_degree=ij_degrees,phase_list=["p","P"])
        ij_pwv_ttime = ij_pwv_ttime_allP[0].time
        
        ## start collecting waveform data 10 seconds before predicted p-wave arrival
        ev_collecttime_all.append(event.origins[0].time + (ij_pwv_ttime - 10))
        ## collect a total of 120 seconds
        ev_endtime_all.append(event.origins[0].time + (ij_pwv_ttime - 10) + 120)
        
        ## append:
        distances_all.append(ij_grcircle_distance_km)
        ev_mag_df_all.append(event.magnitudes[0].mag)
        ev_origintime_df_all.append(event.origins[0].time)
        
    print('st # ' + str(i_station) + ' of ' +str(len(st_stas)) + ' is done')
        
## Save then into a dictionary/dataframe, and sdave to file:
metadata_all = {'distance':distances_all, 'stations':sta_df_all, 'mag':ev_mag_df_all, 
                'origint':ev_origintime_df_all, 'collecttime':ev_collecttime_all, 'endtime':ev_endtime_all,
                'evlat':evdf_lat_all, 'evlon':evdf_lon_all, 'evdepth':evdf_depth_all}

metadata_all_df = pd.DataFrame(metadata_all)
metadata_all_df.to_csv((meta_dir + 'metadata_all.csv'),index=False)

st # 0 of 1185 is done
st # 1 of 1185 is done
st # 2 of 1185 is done
st # 3 of 1185 is done
st # 4 of 1185 is done
st # 5 of 1185 is done
st # 6 of 1185 is done
st # 7 of 1185 is done
st # 8 of 1185 is done
st # 9 of 1185 is done
st # 10 of 1185 is done
st # 11 of 1185 is done
st # 12 of 1185 is done
st # 13 of 1185 is done
st # 14 of 1185 is done
st # 15 of 1185 is done
st # 16 of 1185 is done
st # 17 of 1185 is done
st # 18 of 1185 is done
st # 19 of 1185 is done
st # 20 of 1185 is done
st # 21 of 1185 is done
st # 22 of 1185 is done
st # 23 of 1185 is done
st # 24 of 1185 is done
st # 25 of 1185 is done
st # 26 of 1185 is done
st # 27 of 1185 is done
st # 28 of 1185 is done
st # 29 of 1185 is done
st # 30 of 1185 is done
st # 31 of 1185 is done
st # 32 of 1185 is done
st # 33 of 1185 is done
st # 34 of 1185 is done
st # 35 of 1185 is done
st # 36 of 1185 is done
st # 37 of 1185 is done
st # 38 of 1185 is done
st # 39 of 1185 is done
st # 40 of 1185 is done
st # 41 of 1185 is done
st

st # 333 of 1185 is done
st # 334 of 1185 is done
st # 335 of 1185 is done
st # 336 of 1185 is done
st # 337 of 1185 is done
st # 338 of 1185 is done
st # 339 of 1185 is done
st # 340 of 1185 is done
st # 341 of 1185 is done
st # 342 of 1185 is done
st # 343 of 1185 is done
st # 344 of 1185 is done
st # 345 of 1185 is done
st # 346 of 1185 is done
st # 347 of 1185 is done
st # 348 of 1185 is done
st # 349 of 1185 is done
st # 350 of 1185 is done
st # 351 of 1185 is done
st # 352 of 1185 is done
st # 353 of 1185 is done
st # 354 of 1185 is done
st # 355 of 1185 is done
st # 356 of 1185 is done
st # 357 of 1185 is done
st # 358 of 1185 is done
st # 359 of 1185 is done
st # 360 of 1185 is done
st # 361 of 1185 is done
st # 362 of 1185 is done
st # 363 of 1185 is done
st # 364 of 1185 is done
st # 365 of 1185 is done
st # 366 of 1185 is done
st # 367 of 1185 is done
st # 368 of 1185 is done
st # 369 of 1185 is done
st # 370 of 1185 is done
st # 371 of 1185 is done
st # 372 of 1185 is done


st # 664 of 1185 is done
st # 665 of 1185 is done
st # 666 of 1185 is done
st # 667 of 1185 is done
st # 668 of 1185 is done
st # 669 of 1185 is done
st # 670 of 1185 is done
st # 671 of 1185 is done
st # 672 of 1185 is done
st # 673 of 1185 is done
st # 674 of 1185 is done
st # 675 of 1185 is done
st # 676 of 1185 is done
st # 677 of 1185 is done
st # 678 of 1185 is done
st # 679 of 1185 is done
st # 680 of 1185 is done
st # 681 of 1185 is done
st # 682 of 1185 is done
st # 683 of 1185 is done
st # 684 of 1185 is done
st # 685 of 1185 is done
st # 686 of 1185 is done
st # 687 of 1185 is done
st # 688 of 1185 is done
st # 689 of 1185 is done
st # 690 of 1185 is done
st # 691 of 1185 is done
st # 692 of 1185 is done
st # 693 of 1185 is done
st # 694 of 1185 is done
st # 695 of 1185 is done
st # 696 of 1185 is done
st # 697 of 1185 is done
st # 698 of 1185 is done
st # 699 of 1185 is done
st # 700 of 1185 is done
st # 701 of 1185 is done
st # 702 of 1185 is done
st # 703 of 1185 is done


st # 996 of 1185 is done
st # 997 of 1185 is done
st # 998 of 1185 is done
st # 999 of 1185 is done
st # 1000 of 1185 is done
st # 1001 of 1185 is done
st # 1002 of 1185 is done
st # 1003 of 1185 is done
st # 1004 of 1185 is done
st # 1005 of 1185 is done
st # 1006 of 1185 is done
st # 1007 of 1185 is done
st # 1008 of 1185 is done
st # 1009 of 1185 is done
st # 1010 of 1185 is done
st # 1011 of 1185 is done
st # 1012 of 1185 is done
st # 1013 of 1185 is done
st # 1014 of 1185 is done
st # 1015 of 1185 is done
st # 1016 of 1185 is done
st # 1017 of 1185 is done
st # 1018 of 1185 is done
st # 1019 of 1185 is done
st # 1020 of 1185 is done
st # 1021 of 1185 is done
st # 1022 of 1185 is done
st # 1023 of 1185 is done
st # 1024 of 1185 is done
st # 1025 of 1185 is done
st # 1026 of 1185 is done
st # 1027 of 1185 is done
st # 1028 of 1185 is done
st # 1029 of 1185 is done
st # 1030 of 1185 is done
st # 1031 of 1185 is done
st # 1032 of 1185 is done
st # 1033 of 1185 is done
st # 1034 of 118

In [7]:
######## DOWNLOAD AND SAVE PROCESSED WAVEFORMS ###########
all_metadata_df = pd.read_csv((meta_dir + 'metadata_all.csv'))
#all_metadata_df = pd.read_csv((meta_dir + 'metadata_dldata.csv'))



dlsuccess = [False] * len(all_metadata_df)
for recording_i in range(len(all_metadata_df)):
    try:
        i_st = client.get_waveforms(network="*",station=all_metadata_df.stations[recording_i],location="*",
                                    channel="???",starttime=metadata_all['collecttime'][recording_i],
                                    endtime=metadata_all['endtime'][recording_i],attach_response=True)
        dlsuccess[recording_i] = True
        print(np.str(recording_i) +  ' for station ' + np.str(all_metadata_df.stations[recording_i]) + ' evm ' + np.str(all_metadata_df.mag[recording_i]) + ' downloaded waveforms')
    except:
        dlsuccess[recording_i] = False
        print(np.str(recording_i) + ' has nothing')
    ## if there's data, keep going...
    if dlsuccess[recording_i]==True:
        print('moving on...')
        # remove response - get sensitivity for accelerometer since it's a flat gain:
        for j_channel in range(len(i_st)):
            ij_sensitivity = i_st[j_channel].stats.response.instrument_sensitivity.value
            i_st[j_channel].data = i_st[j_channel].data / ij_sensitivity
        
        ## Remove pre-event baseline mean:
#         for j_channel in range(len(i_st)):
#             ij_5sec_index = np.where(i_st[j_channel].times() <= 5)[0]
#             ij_preeventmean = np.mean(i_st[j_channel].data[ij_5sec_index])
#             i_st[j_channel].data = i_st[j_channel].data - ij_preeventmean
        
        ### save the data....
        i_datetime = UTCDateTime(all_metadata_df.collecttime[recording_i]).datetime
        i_time_for_path = np.str(i_datetime.year) + np.str(i_datetime.month)+ np.str(i_datetime.day)+ np.str(i_datetime.hour)+ np.str(i_datetime.minute)+ np.str(i_datetime.second)+ np.str(i_datetime.microsecond)
        i_mseedpath = mseed_dir + np.str(all_metadata_df.stations[recording_i]) + '_' + np.str(all_metadata_df.mag[recording_i]) + '_' + np.str(np.round(all_metadata_df.distance[recording_i],decimals=2)) + '_' + i_time_for_path + '.mseed'
        i_st.write(i_mseedpath,format='mseed')

## save metadata file with only dlsuccess:
dl_metadata_df = all_metadata_df[dlsuccess]
dl_metadata_df.to_csv((meta_dir + 'metadata_dldata.csv'),index=False)

0 has nothing
1 has nothing
2 has nothing
3 has nothing
4 has nothing
5 has nothing
6 has nothing
7 has nothing
8 has nothing
9 has nothing
10 has nothing
11 has nothing
12 has nothing
13 has nothing
14 has nothing
15 has nothing
16 has nothing
17 has nothing
18 has nothing
19 has nothing
20 has nothing
21 has nothing
22 has nothing
23 has nothing
24 has nothing
25 has nothing
26 has nothing
27 has nothing
28 has nothing
29 has nothing
30 has nothing
31 has nothing
32 has nothing
33 has nothing
34 has nothing
35 has nothing
36 has nothing
37 has nothing
38 has nothing
39 has nothing
40 has nothing
41 has nothing
42 has nothing
43 has nothing
44 has nothing
45 has nothing
46 has nothing
47 has nothing
48 has nothing
49 has nothing
50 has nothing
51 has nothing
52 has nothing
53 has nothing
54 has nothing
55 has nothing
56 has nothing
57 has nothing
58 has nothing
59 has nothing
60 has nothing
61 has nothing
62 has nothing
63 has nothing
64 has nothing
65 has nothing
66 has nothing
67 ha

/var/folders/b9/lpn_b8pj1n7_m6yczj34v_080000gn/T/ipykernel_46492/2053336032.py:17: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print(np.str(recording_i) + ' has nothing')


In [272]:
main_flatfile_path = '/Users/rshimony/Desktop/WillametteValley/eve_4_inv/metadata/full_flatfile.csv'

allmetadata = pd.read_csv(main_flatfile_path)

In [273]:
i_origintime = allmetadata['OrgT']
# i_date,i_time = allmetadata['OrgT'].split(' ')
# i_year,i_month,i_day = i_date.split('-')
# i_hr,i_min,i_sec = i_time.split(':')

In [276]:
i_date,i_time = i_origintime[3].split(' ')
print(i_date)
print(i_time)

i_year,i_month,i_day = i_date.split('-')
print(i_year)
print(i_month)
print(i_day)

2020-12-04
14:07:31
2020
12
04
